#  Import

In [46]:
print("hello")

hello


In [47]:
import os
import sys

import numpy as  np
from lwm_model import lwm  # 클래스 import
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset as TorchDataset, DataLoader, Subset
import time
import torch.nn as nn

from PIL import Image
import numpy as np

from deepverse import ParameterManager
from deepverse.scenario import ScenarioManager
from deepverse import Dataset

from deepverse.visualizers import ImageVisualizer, LidarVisualizer

# Settings

In [48]:
# Scenes 2000
## Subcarriers 64

scenarios_name = "DT31"
config_path = f"scenarios/{scenarios_name}/param/config.m"
param_manager = ParameterManager(config_path)

params = param_manager.get_params()

param_manager.params["scenes"] =list(range(100))
param_manager.params["comm"]["OFDM"]["selected_subcarriers"] = list(range(64))


# Generate a dataset

In [49]:
dataset = Dataset(param_manager)

Generating camera dataset: ⏳ In progress
Generating camera dataset: ✅ Completed (0.00s)
Generating LiDAR dataset: ⏳ In progress
Generating LiDAR dataset: ✅ Completed (0.00s)
Generating mobility dataset: ⏳ In progress
Generating mobility dataset: ✅ Completed (0.00s)
Generating comm dataset: ⏳ In progress


Generating comm dataset: ✅ Completed (1.92s)
Generating radar dataset: ⏳ In progress


Generating radar dataset: ✅ Completed (227.51s)


# location dataset 
 지금 실험에서는 안쓰임

In [50]:
# comm = dataset.comm_dataset
# location =  comm

# location = [
#     {
#         "bs_loc": d["bs_loc"],                      # (3,)
#         "ue_loc": np.asarray(d["ue_loc"]).squeeze() # (3,)  (원래 (1,3)이면 squeeze)
#     }
#     for row in comm.data      # row: [dict] 형태
#     for d in row              # d: dict
# ]

# print(ue_location)  #)

# communication  dataset

In [51]:
# UE 정보
comm = dataset.comm_dataset
ch = comm.data[0][0]['ue'][0]
print(ch.coeffs.shape)  

(1, 16, 64)


# preprocessing

In [52]:
def get_coeffs_from_frame(frame, ue_idx=0):
    ue_obj = frame["ue"]

    # 케이스1) list/tuple이면 ue_idx로 선택
    if isinstance(ue_obj, (list, tuple)):
        ch_obj = ue_obj[ue_idx]
    else:
        # 케이스2) 단일 OFDMChannel이면 그대로 사용
        ch_obj = ue_obj

    # coeffs는 dict key가 아니라 attribute일 확률이 매우 큼
    if hasattr(ch_obj, "coeffs"):
        return ch_obj.coeffs

    # 혹시 dict라면 마지막 보험
    if isinstance(ch_obj, dict) and "coeffs" in ch_obj:
        return ch_obj["coeffs"]

    raise TypeError(f"Cannot get coeffs. ue type={type(ue_obj)}, ch type={type(ch_obj)}")


In [53]:
def get_train_min_max_realimag(frames, train_idx, us_idx=0):

    rmin, rmax =  float('inf'), float('-inf')
    imin, imax =  float('inf'), float('-inf')

    print("Calculating min/max over training set...")

    for t  in train_idx:
        frame  = frames[t]
        coeffs  = get_coeffs_from_frame(frame, us_idx)  # (N_subcarriers, )

        rmin = min(rmin, float(coeffs.real.min()))
        rmax = max(rmax, float(coeffs.real.max()))
        imin = min(imin, float(coeffs.imag.min()))
        imax = max(imax, float(coeffs.imag.max()))

    print(f"Done. rmin={rmin}, rmax={rmax}, imin={imin}, imax={imax}")
    return (rmin, rmax), (imin, imax)

In [54]:
# eps  1e-12 -> 1e-23으로 변경
# 실제 값을 보면 r_min의 경우 −0.0000010915750361793117 이런 값이 됨
def preprocess_channel_coeffs_minmax(coeffs_np, r_min, r_max, i_min, i_max, device="cuda", eps=1e-23):
    # Convert Numpy to Tensor
    coeffs = torch.from_numpy(coeffs_np).to(torch.complex64)
    
    r = coeffs.real
    i = coeffs.imag
    
    # Min-Max Scaling [0, 1]
    # Add eps to denominator to prevent division by zero
    r_scaled = (r - r_min) / max(r_max - r_min, eps)
    i_scaled = (i - i_min) / max(i_max - i_min, eps)
    
    # Concat (Maintains shape like (..., 2*subcarriers))
    H = torch.cat([r_scaled, i_scaled], dim=-1).to(device)
    return H

In [55]:
# 사용예시
H = preprocess_channel_coeffs_minmax(ch.coeffs, r_min=-0.5, r_max=0.5, i_min=-0.5, i_max=0.5)
print(H.shape)  # (1, 16, 128) 64 subcar

torch.Size([1, 16, 128])


# Dataset 구현

In [56]:
def flatten_comm_frames(comm):
    frames = []
    for row in comm.data:
        for d in row:
            frames.append(d)
    return frames

class ChannelNextStepDatasetGPU(TorchDataset):
    def __init__(self, comm_frames, ue_idx=0, past_len=16, device="cuda",
                 r_min=0.0, r_max=1.0, i_min=0.0, i_max=1.0):
        self.comm_frames = comm_frames
        self.ue_idx = ue_idx
        self.past_len = past_len
        self.device = device

        self.r_min, self.r_max = r_min, r_max
        self.i_min, self.i_max = i_min, i_max

        self.N = len(self.comm_frames)
        self.valid_start = past_len - 1
        self.valid_end = self.N - 2

    def __len__(self):
        return self.valid_end - self.valid_start + 1

    def __getitem__(self, idx):
        t = self.valid_start + idx

        # 1) Channel Past
        ch_list = []
        for k in range(t - self.past_len + 1, t + 1):
            coeffs_np = get_coeffs_from_frame(self.comm_frames[k], ue_idx=self.ue_idx)
            h = preprocess_channel_coeffs_minmax(
                coeffs_np,
                self.r_min, self.r_max, self.i_min, self.i_max,
                device=self.device
            ).reshape(-1)
            ch_list.append(h)
        channel_past = torch.stack(ch_list, dim=0)  # (T, F_in)

        # 2) Target (next step)
        coeffs_np_next = get_coeffs_from_frame(self.comm_frames[t + 1], ue_idx=self.ue_idx)
        target = preprocess_channel_coeffs_minmax(
            coeffs_np_next,
            self.r_min, self.r_max, self.i_min, self.i_max,
            device=self.device
        ).reshape(-1)  # (F_out)

        return channel_past, target


# DataLoader 구현

In [57]:
comm_frames = flatten_comm_frames(dataset.comm_dataset)

ds = ChannelNextStepDatasetGPU(
    comm_frames=comm_frames,
    ue_idx=0,
    past_len=15,
    device="cuda"
)

loader = DataLoader(
    ds,
    batch_size=8,
    shuffle=True,
    num_workers=0,     
    pin_memory=False   # ✅ 의미 없음 (이미 GPU)
)

ch, y = next(iter(loader))
print(ch.shape,  y.shape)
print(ch.device, y.device)
    

torch.Size([8, 15, 2048]) torch.Size([8, 2048])
cuda:0 cuda:0


In [58]:
ch, y = next(iter(loader))  # or train_loader

print("ch shape:", ch.shape, "y shape:", y.shape)

# 첫 샘플, 마지막 타임스텝, 앞 20개 값만 보기
print("ch[0, -1, :20] =", ch[0, -1, :20])

# 첫 샘플 타깃 벡터 앞 20개
print("y[0, :20] =", y[0, :20])



ch shape: torch.Size([8, 15, 2048]) y shape: torch.Size([8, 2048])
ch[0, -1, :20] = tensor([ 1.3230e-07,  1.1461e-07,  9.1956e-08,  6.1297e-08,  2.1578e-08,
        -2.5845e-08, -7.7558e-08, -1.2910e-07, -1.7628e-07, -2.1633e-07,
        -2.4846e-07, -2.7380e-07, -2.9455e-07, -3.1292e-07, -3.3013e-07,
        -3.4589e-07, -3.5862e-07, -3.6620e-07, -3.6703e-07, -3.6095e-07],
       device='cuda:0')
y[0, :20] = tensor([-4.2879e-07, -4.0625e-07, -3.7814e-07, -3.4526e-07, -3.0989e-07,
        -2.7518e-07, -2.4410e-07, -2.1836e-07, -1.9763e-07, -1.7952e-07,
        -1.6023e-07, -1.3567e-07, -1.0283e-07, -6.0708e-08, -1.0651e-08,
         4.4064e-08,  9.9264e-08,  1.5113e-07,  1.9726e-07,  2.3710e-07],
       device='cuda:0')


In [59]:
print("ch min/max:", ch.min().item(), ch.max().item())
print("y  min/max:", y.min().item(),  y.max().item())

print("ch mean/std:", ch.mean().item(), ch.std().item())
print("y  mean/std:", y.mean().item(),  y.std().item())


ch min/max: -1.5673900861656875e-06 1.5746156805107603e-06
y  min/max: -9.961363502952736e-07 1.0209263336946606e-06
ch mean/std: -3.420363992034936e-09 4.5454865471583616e-07
y  mean/std: -9.203304784932698e-11 3.6898541111440863e-07


In [60]:
idx = 0
channel_past, target = ds[idx]  # 이미 스케일링 된 텐서

t = ds.valid_start + idx

raw_now  = get_coeffs_from_frame(comm_frames[t], ue_idx=0)      # numpy complex
raw_next = get_coeffs_from_frame(comm_frames[t+1], ue_idx=0)

print("raw_now shape:", raw_now.shape, "dtype:", raw_now.dtype)
print("raw_now real min/max:", raw_now.real.min(), raw_now.real.max())
print("raw_now imag min/max:", raw_now.imag.min(), raw_now.imag.max())

print("raw_next real min/max:", raw_next.real.min(), raw_next.real.max())
print("raw_next imag min/max:", raw_next.imag.min(), raw_next.imag.max())


raw_now shape: (1, 16, 64) dtype: complex128
raw_now real min/max: -1.0915750361793117e-06 1.1047814353805472e-06
raw_now imag min/max: -1.1138026455803887e-06 1.1093295598083908e-06
raw_next real min/max: -1.0288621162405502e-06 1.0310415144220829e-06
raw_next imag min/max: -1.0319768287920318e-06 1.0419675883991902e-06


# Fine-tuning
data shape 맞추기 위해

In [61]:
class LWMForecasterNoEdit(nn.Module):
    """
    lwm 코드는 수정하지 않고, encoder(embedding+layers)만 직접 호출해서 forecasting
    Input : ch  (B,T,F_in)
    Output: yhat(B,F_out)
    """
    def __init__(
            self,      
            base_lwm: nn.Module,
            F_in: int,
            F_out: int,
            pool="last",
            freeze_backbone: bool = False,
            element_length: int = 16,
            d_model: int = 64
            ):
        super().__init__()
        self.base = base_lwm
        self.pool = pool

        n_dim = self.base.embedding.element_length  # = ELEMENT_LENGTH(16)

        self.in_proj = nn.Sequential(
            nn.Linear(F_in, 512),
            nn.GELU(),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, n_dim),
        )
        self.head = nn.Linear(d_model, F_out)

    def forward(self, ch):
        x = self.in_proj(ch)          # (B,T,F_in) -> (B,T,16)

        # ✅ lwm.forward() 쓰지 않고 encoder만 사용
        out = self.base.embedding(x)  # (B,T,64)
        for layer in self.base.layers:
            out, _ = layer(out)       # (B,T,64)

        if self.pool == "last":
            z = out[:, -1, :]
        elif self.pool == "mean":
            z = out.mean(dim=1)
        else:
            raise ValueError(f"Unknown pool={self.pool}")

        return self.head(z)           # (B,F_out)


## NMSE(dB)

In [ ]:
@torch.no_grad()
def nmse_db(yhat: torch.Tensor, y: torch.Tensor, eps: float = 1e-23  ) -> torch.Tensor:
    # yhat, y: (B,F)
    num = torch.sum((yhat - y) ** 2, dim=1)
    den = torch.sum(y ** 2, dim=1).clamp_min(eps)
    nmse = num / den
    return 10.0 * torch.log10(nmse.clamp_min(eps)).mean()


# Train/Val split

In [63]:
n = len(ds)
n_train = int(0.75 * n)
train_idx = list(range(0, n_train))
val_idx = list(range(n_train, n))

train_ts = [ds.valid_start + i for i in train_idx]

# min__max scaler 적용
(real_min,  real_max), (imag_min, imag_max) = get_train_min_max_realimag(
    comm_frames, train_ts, us_idx=0
)

ds.r_min = real_min
ds.r_max = real_max
ds.i_min = imag_min
ds.i_max = imag_max

print("Dataset statistical values set in the dataset.")

train_ds = Subset(ds, train_idx)
val_ds   = Subset(ds, val_idx)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=0)

# Verify
ch, y = next(iter(train_loader))
print("\n=== Data Check ===")
print(f"y stats | min: {y.min().item():}, max: {y.max().item():}")
print("If scaling worked correctly, values should be within [0, 1].")
print("ch shape:", ch.shape, "y shape:", y.shape)   # 추가 확인용

Calculating min/max over training set...
Done. rmin=-1.0915750361793117e-06, rmax=1.1047814353805472e-06, imin=-1.1138026455803887e-06, imax=1.1093295598083908e-06
Dataset statistical values set in the dataset.

=== Data Check ===
y stats | min: 0.01882527768611908, max: 0.9940890073776245
If scaling worked correctly, values should be within [0, 1].
ch shape: torch.Size([32, 15, 2048]) y shape: torch.Size([32, 2048])


In [64]:
len(val_loader)

1

# Model generate and also check

In [65]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# 배치 하나로 F_in/F_out 자동 확정 
ch, y = next(iter(train_loader))
F_in  = ch.shape[-1]
F_out = y.shape[-1]
print("Detected:", "F_in=", F_in, "F_out=", F_out)
print("Batch devices:", ch.device, y.device)

# --- base LWM (lwm_model은 수정 안 함) ---
# 너가 lwm 클래스를 정의해둔 파일에서 import 되어 있어야 함
# from lwm_model import lwm
base = lwm(
    element_length=16,
    d_model=64,
    max_len=129,
    n_layers=12
).to(device)

# 또는 pretrained 쓰면:
# base = lwm.from_pretrained("model_weights.pth", device=device)

# --- forecasting wrapper (fine-tuning 쪽에서만) ---
model = LWMForecasterNoEdit(
    base_lwm=base,
    F_in=F_in,
    F_out=F_out,
    pool="last"
).to(device)

# sanity forward
model.eval()
with torch.no_grad():
    yhat = model(ch.to(device))
print("yhat:", yhat.shape, "y:", y.shape)


device: cuda
Detected: F_in= 2048 F_out= 2048
Batch devices: cuda:0 cuda:0
yhat: torch.Size([32, 2048]) y: torch.Size([32, 2048])


# Train/ Eval 함수 (AMP + grad clip)

In [66]:
def train_one_epoch(model, loader, optimizer, device, use_amp=True, grad_clip=1.0):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    total_loss = 0.0
    total_nmse = 0.0
    n = 0

    for ch, y in loader:
        # Dataset이 이미 cuda 텐서를 반환하더라도 안전하게 유지
        ch = ch.to(device, non_blocking=True)
        y  = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=use_amp):
            yhat = model(ch)
            loss = F.mse_loss(yhat, y)

        scaler.scale(loss).backward()

        if grad_clip is not None and grad_clip > 0:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        total_nmse += nmse_db(yhat.detach(), y).item()
        n += 1

    return total_loss / max(n, 1), total_nmse / max(n, 1)


@torch.no_grad()
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()

    total_loss = 0.0
    total_nmse = 0.0
    n = 0

    for ch, y in loader:  
        ch = ch.to(device, non_blocking=True)
        y  = y.to(device, non_blocking=True)

        yhat = model(ch)  
        loss = F.mse_loss(yhat, y)

        total_loss += loss.item()
        total_nmse += nmse_db(yhat, y).item()
        n += 1

    return total_loss / max(n, 1), total_nmse / max(n, 1)


#  Optiimizer  / Scheduler 설정

In [67]:
# requires_grad=True인 파라미터만 학습
trainable_params = [p for p in model.parameters() if p.requires_grad]
print("trainable params:", sum(p.numel() for p in trainable_params))

optimizer = torch.optim.AdamW(trainable_params, lr=1e-3, weight_decay=1e-4)

# (선택) cosine scheduler
epochs = 1000
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)


trainable params: 1853648


# 학습 루프 + checkpoint 저장

In [68]:
import time
import torch

best_val_loss = float("inf")
best_val_nmse = None
best_epoch = -1

t_train0 = time.time()  # ✅ 전체 학습 시작 시간

for epoch in range(1, epochs + 1):
    t0 = time.time()

    tr_loss, tr_nmse = train_one_epoch(
        model, train_loader, optimizer,
        device=device, use_amp=True, grad_clip=1.0
    )
    va_loss, va_nmse = evaluate(model, val_loader, device=device)

    scheduler.step()

    dt = time.time() - t0
    print(
        f"[{epoch:02d}/{epochs}] "
        f"train loss={tr_loss:.6f}, nmse(dB)={tr_nmse:.4f} | "
        f"val loss={va_loss:.6f}, nmse(dB)={va_nmse:.4f} | "
        f"{dt:.1f}s"
    )

    # ✅ best 체크포인트 저장 + best_nmse/epoch 기록
    if va_loss < best_val_loss:
        best_val_loss = va_loss
        best_val_nmse = va_nmse
        best_epoch = epoch

        torch.save(
            {
                "epoch": epoch,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "F_in": F_in,
                "F_out": F_out,
                "best_val_loss": best_val_loss,   # (선택) 같이 저장
                "best_val_nmse": best_val_nmse,   # (선택) 같이 저장
            },
            "best_finetune.pt"
        )
        print("  ↳ saved best_finetune.pt")

# ✅ 루프가 끝난 뒤 요약 출력 (스샷 느낌)
total_sec = time.time() - t_train0
print("\n=== Done ===")
print(f"Best val loss     : {best_val_loss:.6f}")
print(f"Best val NMSE (dB): {best_val_nmse:.4f}")
print(f"Best epoch        : {best_epoch}")
print(f"Total train time  : {total_sec/60:.2f} min ({total_sec:.1f} sec)")


/tmp/ipykernel_30651/578535649.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
/tmp/ipykernel_30651/578535649.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


[01/1000] train loss=0.597673, nmse(dB)=3.2387 | val loss=0.420491, nmse(dB)=2.1047 | 0.2s
  ↳ saved best_finetune.pt
[02/1000] train loss=0.428028, nmse(dB)=1.7726 | val loss=0.303385, nmse(dB)=0.6865 | 0.2s
  ↳ saved best_finetune.pt
[03/1000] train loss=0.326283, nmse(dB)=0.5874 | val loss=0.233207, nmse(dB)=-0.4567 | 0.2s
  ↳ saved best_finetune.pt
[04/1000] train loss=0.273938, nmse(dB)=-0.1829 | val loss=0.195118, nmse(dB)=-1.2318 | 0.2s
  ↳ saved best_finetune.pt
[05/1000] train loss=0.243138, nmse(dB)=-0.7075 | val loss=0.168227, nmse(dB)=-1.8766 | 0.2s
  ↳ saved best_finetune.pt
[06/1000] train loss=0.217001, nmse(dB)=-1.2101 | val loss=0.141138, nmse(dB)=-2.6404 | 0.2s
  ↳ saved best_finetune.pt
[07/1000] train loss=0.187942, nmse(dB)=-1.8343 | val loss=0.115264, nmse(dB)=-3.5221 | 0.2s
  ↳ saved best_finetune.pt
[08/1000] train loss=0.159499, nmse(dB)=-2.5733 | val loss=0.091804, nmse(dB)=-4.5143 | 0.2s
  ↳ saved best_finetune.pt
[09/1000] train loss=0.133922, nmse(dB)=-3.33

In [69]:
# 채널-only: (ch, y)만 받음
ch, y = next(iter(train_loader))

print("y abs mean:", y.abs().mean().item())
print("y abs max :", y.abs().max().item())
print("y power   :", (y**2).mean().item())

with torch.no_grad():
    yhat = model(ch.to(device))   # img 제거

print("yhat abs mean:", yhat.abs().mean().item())
print("yhat abs max :", yhat.abs().max().item())
print("yhat power   :", (yhat**2).mean().item())


y abs mean: 0.4907627999782562
y abs max : 0.9797839522361755
y power   : 0.28452348709106445
yhat abs mean: 0.48926734924316406
yhat abs max : 1.0784201622009277
yhat power   : 0.2785642147064209


# 데이터 입력 및 형태

In [70]:
print("=== dataset sizes ===")
print("N(comm_frames):", len(comm_frames))
print("past_len      :", ds.past_len)
print("len(ds)       :", len(ds))
print("len(train_ds) :", len(train_ds))
print("len(val_ds)   :", len(val_ds))
print("len(train_loader):", len(train_loader))
print("len(val_loader)  :", len(val_loader))

print("\n=== one batch shapes ===")
ch, y = next(iter(train_loader))  # ✅ (ch, y)만 받기
print("ch :", tuple(ch.shape), " -> (B,T,F_in)")
print("y  :", tuple(y.shape), " -> (B,F_out)")

with torch.no_grad():
    yhat = model(ch.to(device))   # ✅ model(ch)만 호출

print("yhat:", tuple(yhat.shape), " -> (B,F_out)")
print("this forward predicted vectors:", yhat.shape[0], "(=B)")
print("each vector predicts elements:", yhat.shape[1], "(=F_out)")


=== dataset sizes ===
N(comm_frames): 100
past_len      : 15
len(ds)       : 85
len(train_ds) : 63
len(val_ds)   : 22
len(train_loader): 2
len(val_loader)  : 1

=== one batch shapes ===
ch : (32, 15, 2048)  -> (B,T,F_in)
y  : (32, 2048)  -> (B,F_out)
yhat: (32, 2048)  -> (B,F_out)
this forward predicted vectors: 32 (=B)
each vector predicts elements: 2048 (=F_out)
